In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
def show_images(images, str_title):
    #images는 무조건 n^2으로 가정
    num_images = images.shape[0]
    fig = plt.figure(figsize=(8, 8))
    fig.suptitle(str_title, fontsize=24)

    for ind in range(0, num_images):
        plt.subplot(np.sqrt(num_images),
                    np.sqrt(num_images), ind + 1)
        plt.imshow(images[ind, :, :], cmap='gray')
        plt.axis('off')

In [3]:
num_displays_per_row = 5
num_displays = num_displays_per_row**2

In [4]:
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train[..., np.newaxis] # 이게 더 빠름
# X_train = np.expand_dims(X_train, axis=3)
X_test = X_test[..., np.newaxis]

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

X_train /= 255.0
X_test /= 255.0

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


In [5]:
num_epochs = 10
size_batch = 64
rate_learning_generator = 1e-4
rate_learning_discriminator = 1e-4
dim_noise = 32 #G 차원

In [6]:
dataset_train = tf.data.Dataset.from_tensor_slices(X_train)
dataset_train = dataset_train.shuffle(10000).batch(size_batch)

dataset_test = tf.data.Dataset.from_tensor_slices(X_test)
dataset_test = dataset_test.batch(size_batch)

In [13]:
model_generator = tf.keras.Sequential([
    tf.keras.layers.Dense(7*7*256, input_shape = (dim_noise, )),  #(bs, 7*7*256)
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Reshape((7, 7, 256)), #(bs, 7, 7, 256)
    tf.keras.layers.Conv2DTranspose(128, (5, 5), strides = (1, 1), padding = 'same'), #(bs, 7, 7, 128)
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Conv2DTranspose(64, (5, 5), strides = (2, 2), padding = 'same'), #(bs, 14, 14, 64)
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Conv2DTranspose(1, (5, 5), strides = (2, 2), padding = 'same', activation = tf.nn.tanh)
    
])

model_discriminator = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (5, 5), strides = (2, 2), padding = 'same', input_shape = [28, 28, 1]), #(bs, 14, 14, 64)
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, (5, 5), strides = (2, 2), padding = 'same'), #(bs, 7, 7, 128)
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(), #(bs, 7 * 7 * 128)
    tf.keras.layers.Dense(1) # 리얼 데이터인지 구별하는거라(진짜인지 아닌지 판별) -> binary classification
    
])

In [9]:
loss_ce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def loss_discriminator(outputs_real, outputs_fake):
    # outputs_real: shape = (64(batch size), 1) - > real probability
    # outputs_fake: shape = (64(batch size), 1) - > real probability
    loss_1 = loss_ce(tf.ones_like(outputs_real), outputs_real) # 1로만 채워진 matrix, shape이 outputs real과 동일하도록
    loss_2 = loss_ce(tf.zeros_like(outputs_fake), outputs_fake)
    loss_all = loss_1 + loss_2
    return loss_all

def loss_generator(outputs_fake):
    return loss_ce(tf.ones_like(outputs_fake), outputs_fake)

In [10]:
optimizer_gen = tf.keras.optimizers.Adam(rate_learning_generator)
optimizer_dis = tf.keras.optimizers.Adam(rate_learning_discriminator)

In [19]:
@tf.function
def step_train(images):
    #images: 실제 이미지 (batch size, 28, 28, 1)
    noise = tf.random.normal([size_batch, dim_noise]) # 정규분포
    with tf.GradientTape() as tape_gen, tf.GradientTape() as tape_dis:
        images_gen = model_generator(noise, training = True) #(batch size, 28, 28, 1)
        outputs_real = model_discriminator(images, training = True) #real probability(batch size, 1)
        outputs_fake = model_discriminator(images_gen, training = True) #real probability(batch size, 1)
        
        loss_dis = loss_discriminator(outputs_real, outputs_fake)
        loss_gen = loss_generator(outputs_fake)
        
    grad_gen = tape_gen.gradient(loss_gen, model_discriminator.trainable_weights)
    grad_dis = tape_dis.gradient(loss_dis, model_discriminator.trainable_weights)
    optimizer_dis.apply_gradients(zip(grad_dis, model_discriminator.trainable_weights))
    optimizer_gen.apply_gradients(zip(grad_gen, model_discriminator.trainable_weights))

    return loss_gen, loss_dis

@tf.function
def step_test(images):
    noise = tf.random.normal([size_batch, dim_noise])
    images_gen = model_generator(noise, training = False) #(batch size, 28, 28, 1)
    outputs_real = model_discriminator(images, training = False) #real probability(batch size, 1)
    outputs_fake = model_discriminator(images_gen, training = False) #real probability(batch size, 1)
        
    loss_dis = loss_discriminator(outputs_real, outputs_fake)
    loss_gen = loss_generator(outputs_fake)

    return loss_gen, loss_dis

@tf.function
def generate_images(num_displays):
    noise = tf.random.normal([num_displays, dim_noise])
    images_gen = model_generator(noise, training = False)

    return images_generated

def evaluate_dataset(dataset, training):
    loss_gen = 0.0
    loss_dis = 0.0
    num_data = 0.0

    for images_batch in dataset:
        if training:
            loss_gen_, loss_dis_ = step_train(images_batch)
        else:
            loss_gen_, loss_dis_ = step_test(images_batch)

        loss_gen += loss_gen_.numpy() * images_batch.shape[0]
        loss_dis += loss_dis_.numpy() * images_batch.shape[0]
        num_data += images_batch.shape[0]

    loss_gen /= num_data
    loss_dis /= num_data

    return loss_gen, loss_dis

In [20]:
for ind_epoch in range(0, num_epochs):
    loss_gen_train, loss_dis_train = evaluate_dataset(dataset_train, True)
    print('[TRAIN] {} epoch: loss_gen {:.4f} loss_disc {:.4f}'.format(ind_epoch + 1,
        loss_gen_train, loss_dis_train))

    loss_gen_test, loss_disc_test = evaluate_dataset(dataset_test, False)
    print('[TEST] loss_gen {:.4f} loss_disc {:.4f}'.format(loss_gen_test,
                                                           loss_dis_test))

    if ind_epoch > 80:
        images_generated = generate_images(num_displays)
        images_generated = tf.squeeze(images_generated, axis=3)

        show_images(images_generated, 'Generated by GAN')
        plt.show()

[TRAIN] 1 epoch: loss_gen 0.5300 loss_disc 0.9101


NameError: name 'loss_dis_test' is not defined